In [1]:
import openai
from typing import List, Dict
import json
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv() 

class SupportAssistant:
    def __init__(self):
        # openai.api_key = api_key
        self.conversation_history = []
        self.current_ticket = {}
        self.client = OpenAI() 
        
    def add_to_history(self, role: str, content: str):
        self.conversation_history.append({"role": role, "content": content})
    
    def get_completion(self, prompt: str) -> Dict:
        self.add_to_history("user", prompt)
        
        messages = [
            {"role": "system", 
            "content": """You are a helpful support assistant. 
            Classify user requests into: Bug, Feature Request, or Other. 
            Collect required information based on the type:
            - Bug: username, description, version
            - Feature Request: username, description, priority
            - Other: username, description.
            
            Return the classification and details in JSON format.
            If all of the required information is provided return with {"end_conversation": true}."""},
            *self.conversation_history
        ]
        
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.2,
            max_tokens=150
        )
        
        raw_response = response.choices[0].message.content
        self.add_to_history("assistant", raw_response)

        # Try to parse JSON safely
        try:
            parsed_response = json.loads(raw_response)
        except json.JSONDecodeError:
            parsed_response = {"message": raw_response, "end_conversation": False}

        return parsed_response

    def create_ticket(self, category: str, details: Dict) -> Dict:
        ticket = {
            "category": category,
            "timestamp": datetime.now().isoformat(),
            "status": "open",
            **details
        }
        return ticket

# Example usage
def main():
    assistant = SupportAssistant()
    print("Support Assistant: Hello! How can I help you today?")
    
    while True:
        user_input = input("User: ")
        if user_input.lower() == "exit":
            break
            
        response = assistant.get_completion(user_input)
        if response.get("end_conversation"):
            print("Support Assistant: Thank you for the information. Your ticket has been created.")
            print("Generated Ticket:", json.dumps(response, indent=2))  # Show ticket details
            break
        
        # If no end_conversation, show assistant message
        print(f"Support Assistant: {response.get('message', response)}")



In [ ]:
main()

Support Assistant: Hello! How can I help you today?
Support Assistant: What feature would you like to request? Please provide a description and the priority of this feature.
Support Assistant: {'classification': 'Feature Request', 'details': {'username': 'herooo', 'description': 'I want dark mode.', 'priority': 'medium'}}
Support Assistant: Great! If you have any more requests or need further assistance, feel free to reach out. Have a wonderful day! 

{"end_conversation": true}
